In [58]:
import pandas as pd
import openreview
import json
from deepdiff import DeepDiff
from pathlib import Path

from collections import OrderedDict
from typing import Any, Callable, Iterable, List, Dict, Optional
import openreview.api as openreview_api

In [ ]:
def save_json(data,fileName,convert = True):
    with open(f"{fileName}.json","w") as file:
        data_converted = data.to_json() if convert else data
        json.dump(data_converted,file,indent=4)

In [109]:
# ai code
def save_json(data: Any, file_name: str, convert: bool = True) -> None:
    def convert_item(item):
        if hasattr(item, "to_json"):
            payload = dict(item.to_json())
            if getattr(item, "details", None) is not None:
                payload["details"] = item.details
            return payload
        if isinstance(item, list):
            return [convert_item(x) for x in item]
        if isinstance(item, dict):
            return {k: convert_item(v) for k, v in item.items()}
        return item

    serializable = convert_item(data) if convert else data
    with open(f"{file_name}.json", "w") as fh:
        json.dump(serializable, fh, indent=4)


In [116]:
# V1 
# url = "ICLR.cc/2017/conference/-/submission"
url = "ICLR.cc/2018/Conference/-/Blind_Submission"
client = openreview.Client(baseurl='https://api.openreview.net')  # API v1
# submissions = client.get_all_notes(invitation=url,details="directReplies;revisions")
submissions = client.get_all_notes(invitation=url,details="revisions")
print(len(submissions))
print(submissions[0].details)
save_json(submissions[0],"rev2")

935
{'revisions': True, 'invitation': {'reply': {'readers': {'description': 'The users who will be allowed to read the above content.', 'values': ['everyone']}, 'writers': {'values': ['ICLR.cc/2018/Conference']}, 'signatures': {'description': 'How your identity will be displayed with the above content.', 'values': ['ICLR.cc/2018/Conference']}, 'content': {'pdf': {'required': True, 'order': 9, 'value-regex': '.*', 'description': 'Upload a PDF file that ends with .pdf'}, 'title': {'required': True, 'order': 1, 'description': 'Title of paper.', 'value-regex': '.{1,250}'}, 'abstract': {'required': True, 'order': 8, 'description': 'Abstract of paper.', 'value-regex': '[\\S\\s]{1,5000}'}, 'authors': {'required': True, 'order': 2, 'values-regex': '[^;,\\n]+(,[^,\\n]+)*', 'description': 'Comma separated list of author names. Please provide real names; identities will be anonymized.'}, 'keywords': {'order': 6, 'values-regex': '(^$)|[^;,\\n]+(,[^,\\n]+)*', 'description': 'Comma separated list of

In [126]:
note = client.get_all_notes(id='rJT0QeZ0-', details='revisions,invitation')
save_json(note,"note")

OpenReviewException: {'name': 'ForbiddenError', 'message': 'User Guest does not have permission to see Note rJT0QeZ0-', 'status': 403, 'details': {'user': 'guest_1761515559664', 'reqId': '2025-10-26-6805830'}}

In [105]:
# V2
url = "ICLR.cc/2017/conference/-/submission"
# url = "ICLR.cc/2016/workshop/-/submission"
v2 = openreview_api.OpenReviewClient(baseurl="https://api2.openreview.net")
sub2 = client.get_all_notes(invitation=url)
print(len(sub2))

490


In [108]:
client = openreview_api.OpenReviewClient(baseurl="https://api2.openreview.net")
invs = client.get_all_invitations(prefix="ICLR.cc/2016/workshop",limit=1)
# blind = client.get_all_invitations(super="ICLR.cc/2023/Conference",regex=".*Blind_Submission")

len(invs)


TypeError: OpenReviewClient.get_all_invitations() got an unexpected keyword argument 'limit'

In [37]:
for s1, s2 in zip(submissions,sub2):
    s1 = s1.to_json()
    s2 = s2.to_json()

    if s1 != s2:
        print("Wrong")
        break
print("Everythin matches")

Everythin matches


In [56]:
s1 = str(submissions[0])
s1 = (submissions[0]).to_json()

# s1 = (submissions[0])
print(type(s1))
with open("test.json","w") as f:
    f.write(s1)
    # json.dump(s1.to_json(),f)

<class 'str'>


In [62]:
s1 = str((submissions[0]))
Path("note.json").write_text(s1, encoding="utf-8")

176112

In [66]:
save_json(submissions[70], "newww")

In [43]:
save_json(submissions[0],"v1_p0dddd")
# save_json(sub2[0],"v2_p0")
print(submissions[0].details)

{'directReplies': [{'id': 'MXV1TACjo_', 'original': None, 'number': 1, 'cdate': 1666251648361, 'mdate': None, 'ddate': None, 'tcdate': 1666251648361, 'tmdate': 1669196188530, 'tddate': None, 'forum': 'zzqBoIFOQ1', 'replyto': 'zzqBoIFOQ1', 'invitation': 'ICLR.cc/2023/Conference/Paper3283/-/Official_Review', 'content': {'confidence': '4: You are confident in your assessment, but not absolutely certain. It is unlikely, but not impossible, that you did not understand some parts of the submission or that you are unfamiliar with some pieces of related work.', 'summary_of_the_paper': 'This paper deals with safe exploration in reinforcement learning in which an agent is required to ensure safety during training. The authors present a neuro-symbolic approach called SPICE. based on symbolic weakest preconditions. Empirically, they evaluate their approach on toy benchmarks, and show that it is able to achieve comparable performance to existing safe learning techniques while incurring fewer safety